In [1]:
# Refer from https://github.com/leeyeehoo/CSRNet-pytorch/blob/master/make_dataset.ipynb
import os
import cv2
import glob
import h5py
from scipy.io import loadmat
import numpy as np
from tqdm import tqdm
from utils import get_density_map_gaussian

Using TensorFlow backend.


In [2]:
root = 'data/ShanghaiTech/'

In [3]:
part_A_train = os.path.join(root, 'part_A/train_data', 'images')
part_A_test = os.path.join(root, 'part_A/test_data', 'images')
part_B_train = os.path.join(root, 'part_B/train_data', 'images')
part_B_test = os.path.join(root, 'part_B/test_data', 'images')
path_sets_train = [part_A_train, part_B_train]
path_sets_test = [part_A_test, part_B_test]

In [4]:
img_paths_train = []
for path in path_sets_train:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths_train.append(img_path)
print(len(img_paths_train))
img_paths_test = []
for path in path_sets_test:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths_test.append(img_path)
print(len(img_paths_test))

700
498


In [5]:
for part in ['A', 'B']:
    for t in ['train', 'test']:
        if not os.path.exists('data/ShanghaiTech/part_{}/{}_data/ground/'.format(part, t)):
            os.mkdir('data/ShanghaiTech/part_{}/{}_data/ground/'.format(part, t))
for img_paths in [img_paths_train, img_paths_test]:
    for img_path in tqdm(img_paths):

        pts = loadmat(img_path.replace('.jpg', '.mat').replace('images', 'ground-truth').replace('IMG_', 'GT_IMG_'))
        img = cv2.imread(img_path)

        k = np.zeros((img.shape[0], img.shape[1]))

        gt = pts["image_info"][0, 0][0, 0][0]

        for i in range(len(gt)):
            if int(gt[i][1]) < img.shape[0] and int(gt[i][0]) < img.shape[1]:
                k[int(gt[i][1]), int(gt[i][0])] = 1

        k = get_density_map_gaussian(k, gt, adaptive_kernel=True)

        file_path = img_path.replace('.jpg', '.h5').replace('images', 'ground')

        with h5py.File(file_path, 'w') as hf:
                hf['density'] = k

100%|██████████| 498/498 [00:41<00:00, 12.07it/s]
